# Extracting the morphometric statistics


**Please make sure to preprocess the raw-reconstructions using the preprocess-morph-SWC-files.ipynb before running this notebook**


In [1]:
import pandas as pd
import numpy as np
import os

from neurontree import NeuronTree as nt
from neurontree.utils import angle_between
import networkx as nx
import copy

from scipy.stats import wasserstein_distance
from itertools import combinations,permutations

#PLOTTING
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:

save_path = "../data/processed/morph/features/" # <-- add your path here if it differs
path_to_reconstructions='../data/processed/morph/nt/' # <-- add your path here if it differs

In [6]:
meta_data_file_path = "../data/m1_patchseq_meta_data.csv"  # <-- add your path here if it differs
cells = pd.read_csv(meta_data_file_path, sep='\t', index_col=0)
cells = cells[cells['Traced'] == 'y']

In [4]:
def get_perc_above_below_overlap(profile_a, profile_b):
    
    profile_a = np.hstack((np.array([0]),profile_a,np.array([0])))
    profile_b = np.hstack((np.array([0]),profile_b,np.array([0])))
    a = np.where(profile_a > 0)[0]
    b = np.where(profile_b > 0)[0]
    
    perc_a_above_b = np.sum(profile_a[:b[0]])/np.sum(profile_a)
    perc_a_below_b = np.sum(profile_a[b[-1]+1:])/np.sum(profile_a)
    perc_a_overlap_b  = 1 - perc_a_above_b - perc_a_below_b
    
    return (perc_a_above_b,perc_a_below_b,perc_a_overlap_b)

def get_longest_neurite(R):
    
    r = R.get_root()

    stem_ids = [s[1] for s in R.edges() if s[0] == r]
    
    neurite_lengths = dict(zip(stem_ids,[0]*len(stem_ids)))
    neurite_paths = dict()
    G = R.get_graph()
    
    # get the path length and the path of each neurite extending from the soma
    for t in R.get_tips():
        path_length = nx.dijkstra_path_length(G,r,t,weight='path_length')
        path = nx.dijkstra_path(G,r,t)

        stem_ix = path[1]
        neurite_lengths[stem_ix] += path_length 
        if stem_ix in neurite_paths.keys():
            neurite_paths[stem_ix] += path
        else:
            neurite_paths[stem_ix] = path
            
    keys = list(neurite_lengths.keys())
    values = list(neurite_lengths.values())
    argix = np.argmax(values)
    
    #get subgraph with all nodes
    subgraph = nx.subgraph(G,set(neurite_paths[keys[argix]]))
    
    return nt.NeuronTree(graph=subgraph)


def get_morphometrics(T, item):

    
    z = dict()
    depth = item['Soma depth (µm)']
    thickness = item['Cortical thickness (µm)']
    z['normalized depth'] = depth/thickness
   
    for part in ['axon', 'dendrite']:

        if part == 'axon':
            T = N.get_axonal_tree()
        elif part == 'dendrite':
            T = N.get_dendritic_tree() 


        if len(T.nodes()) > 5:

            z[part+ ' branch points'] = T.get_branchpoints().size
            extend = T.get_extend()

            z[part + ' width'] = extend[0]
            z[part + ' depth'] = extend[1]
            z[part + ' height'] = extend[2]

            robust_extend = T.get_extend(robust=True)
            z[part + ' robust width'] = robust_extend[0]
            z[part + ' robust depth'] = robust_extend[1]
            z[part + ' robust height'] = robust_extend[2]

            pos = np.array(list(T.get_node_attributes('pos').values()))
            bias = np.max(pos,axis=0) + np.min(pos, axis=0)

            z[part + ' x-bias'] = np.abs(bias[0])
            z[part + ' z-bias'] = bias[2]

            z[part + ' tips'] = T.get_tips().size

            z[part + ' total length'] = np.sum(list(T.get_edge_attributes('euclidean_dist').values()))

            z[part + ' max path distance to soma'] = np.max(list(T.get_path_length().values()))
            z[part + ' max branch order'] = np.max(list(T.get_branch_order().values()))

            path_angles = []
            for p1 in T.get_path_angles().items():
                if p1:
                    path_angles += [p1[1]]

            z[part + ' max path angle'] = np.percentile(path_angles,99.5)
            z[part + ' median path angle'] = np.median(path_angles)

            R = T.get_topological_minor()
            
            # maximal segment path length
            z[part + ' max segment length'] = np.max(list(R.get_segment_length().values()))

            tortuosity = [e[2]['path_length'] / e[2]['euclidean_dist'] for e in R.edges(data=True)]

            z[part + ' log max tortuosity'] = np.log(np.percentile(tortuosity,99.5))
            z[part + ' log min tortuosity'] = np.log(np.min(tortuosity))
            z[part + ' log median tortuosity'] = np.log(np.median(tortuosity))

            branch_angles = list(R.get_branch_angles().values())
            if branch_angles:
                z[part + ' max branch angle'] = np.max(branch_angles)
                z[part + ' min branch angle'] = np.min(branch_angles)
                z[part + ' mean branch angle'] = np.mean(branch_angles)
            else:
                z[part + ' max branch angle'] = np.nan
                z[part + ' min branch angle'] = np.nan
                z[part + ' mean branch angle'] = np.nan


            # z-profiles
            resampled_nodes = T.resample_nodes(d=1)
            z_profile, _ = np.histogram(-1*((resampled_nodes[:,2] - depth)/thickness), 
                                            bins=20, range=[0,1], density=True)
            soma_centered_z_profile, _ = np.histogram(((resampled_nodes[:,2])/thickness),
                                                      bins=81, range=[-1,1], density=True)
            
            z[part + ' z-profile'] = z_profile
            z[part + ' soma-centered z-profile'] = [soma_centered_z_profile]

            z[part + ' above soma'] = np.sum(resampled_nodes[:,2]>0)/resampled_nodes[:,2].shape[0]

            radii = R.get_node_attributes('radius')
            edges = R.edges()
            r = R.get_root()

            z['soma radius'] = radii[int(r)]

            if part == 'axon':

                # get thickness of initial segments
                node_ids = [e[1] for e in edges if (e[0] == r)]
                initial_segments_radius = [radii[int(n)] for n in node_ids]
                z['mean initial segment radius'] = np.mean(initial_segments_radius)

            # get mean neurite thickness
            radii.pop(r)  # remove the soma as it is usually the thickest
            z[part + ' mean neurite radius'] = np.mean(list(radii.values()))

        
            ec = []
            G = R.get_graph()
            for p in np.concatenate((R.get_tips(),R.get_branchpoints())):
                ec.append(np.sqrt(np.sum(G.node[p]['pos']**2)))

            z[part + ' max Euclidean dist'] = np.max(ec)


            # get bifurcation moments
            branch_point_positions = [R.get_graph().node[k]['pos'] for k in R.get_branchpoints()]
            if branch_point_positions:
                z[part + ' first bifurcation moment'] = np.mean(branch_point_positions, axis=0)[2]
                z[part + ' bifurcation standard deviation'] = np.std(branch_point_positions, axis=0)[2]


            if part == 'dendrite':

                stems = [R.get_graph().node[k[1]]['pos'] for k in R.edges(R.get_root())]
                # only calculated in xz plane in degree
                stem_exit_angles = np.array([angle_between([0,-1],s[[0,2]])/np.pi*180 for s in stems])

                # stems
                z['stems'] = len(stems)

                # stem exit histogram
                z['stems exiting up'] = np.sum(stem_exit_angles < 45 )/len(stems)
                z['stems exiting down'] = np.sum(stem_exit_angles > 135 )/len(stems)
                z['stems exiting to the sides']= np.sum((stem_exit_angles>=45) & (stem_exit_angles <= 135))/len(stems)

                # now get morphometrics for longest dendrite
                L = get_longest_neurite(R)
                
                # get branch point positions for apical
                bpp_L = [L.get_graph().node[k]['pos'] for k in L.get_branchpoints()]
                
                path_length = nx.single_source_dijkstra_path_length(L.get_graph(), source=L.get_root(),weight='path_length')
                # get furthes node and the line to it from soma. Furthest in terms of path length.

                G = L.get_graph()
                tips = L.get_tips()
                pl = [path_length[t] for t in tips]
                

                farthest_tip = tips[np.argmax(pl)]
                farthest_tip_pos = G.node[farthest_tip]['pos']
                max_pl = np.max(pl)

                proj_bp = [np.dot(bpp,farthest_tip_pos)/np.linalg.norm(farthest_tip_pos)/max_pl
                           for bpp in bpp_L]

                if proj_bp:
                    # mean bifurcation distance
                    z['"apical" mean bifurcation distance'] = np.mean(proj_bp)

                    # std bifurcation distance
                    z['"apical" std bifurcation distance'] = np.std(proj_bp)

    
                # number of outer bifurcations
                ec = []
                for t in tips:
                    ec.append(np.sqrt(np.sum(G.node[t]['pos']**2)))

                max_ec = np.max(ec)

                outer_bifurcations = 0
                for bp in L.get_branchpoints():
                    if np.sqrt(np.sum(G.node[bp]['pos']**2)) > max_ec/2:
                        outer_bifurcations += 1
                        
                z['"apical" log1p number of outer bifurcations'] = np.log(outer_bifurcations + 1) 
                
                z['"apical" height'] = L.get_extend()[2]
                z['"apical" width'] = L.get_extend()[0]
                
                z['"apical" robust height'] = L.get_extend(robust=True)[2]
                z['"apical" robust width'] = L.get_extend(robust=True)[0]
                
                z['"apical" total length'] = np.sum(list(L.get_edge_attributes('path_length').values()))
                z['"apical" branch points'] = len(L.get_branchpoints())
        else:
            print("No %s recovered"%part)

        # get the overlap and earth mover's distance here
        # overlap
        for key1, key2 in permutations(['axon z-profile', 'dendrite z-profile'],2):
            try:
                profile_a = z[key1]
                profile_b = z[key2]

                above, below, overlap = get_perc_above_below_overlap(profile_a,profile_b)
                z['Log1p fraction of %s above %s'%(key1.split(" ")[0], key2.split(" ")[0])]= np.log(1+above)
                z['Log1p fraction of %s below %s'%(key1.split(" ")[0], key2.split(" ")[0])]= np.log(1+below)
               
            except KeyError:
                continue


        # earth mover's distance
        for key1, key2 in combinations(['axon z-profile', 'dendrite z-profile'],2):
            try:
                profile_a = z[key1]
                profile_b = z[key2]

                z['EMD %s %s'%(key1.split(" ")[0], key2.split(" ")[0])] = wasserstein_distance(profile_a,profile_b)
            except KeyError:
                continue


    # make the arrays a list
    for key in ['axon z-profile', 'dendrite z-profile']:
        try:
            z[key] = [z[key]]
        except KeyError:
            continue
    return z

In [5]:

for rn, item in list(cells.iterrows()):

    file_name = item['Cell']
            
    if not os.path.exists(save_path + file_name + '.csv'):
        
        print('%i: Calculating morphometric statistics for %s' % (rn,file_name))
        
        
        # load in data
        swc = pd.read_csv(path_to_reconstructions + file_name +".swc", 
                          delim_whitespace=True, comment='#',
                              names=['n', 'type', 'x', 'y', 'z', 'radius', 'parent'], index_col=False)
        # create a neurontree
        N = nt.NeuronTree(swc=swc)

        z = dict()
        z['cell id'] = file_name
        
        # get the morphometrics
        d = get_morphometrics(N, item)
        
        z.update(d)
        
        # save data 
        morphometry_data = pd.DataFrame(z)
        morphometry_data.to_csv(save_path+ file_name + ".csv")        
    else:
        continue

# Explore morphometric features



In [6]:
data_path = save_path

# load in all morphometrics files into one data frame
morphometrics = pd.DataFrame()
root, _, files = list(os.walk(data_path))[0]
for f in files:
    temp = pd.read_csv(data_path+f, index_col=0)
    morphometrics = morphometrics.append(temp)

morphometrics = morphometrics.reset_index()
del morphometrics['index']

In [50]:
full_idx = list(morphometrics.columns)
full_idx.remove('cell id')
full_idx.remove('dendrite z-profile')
full_idx.remove('axon z-profile')
full_idx.remove('axon soma-centered z-profile')
full_idx.remove('dendrite soma-centered z-profile')
len(full_idx)

75

In [101]:
#create indices for excitatory and inhibitory cells
excitatory_index = (cells['RNA family'] == 'CT') | (cells['RNA family'] == 'IT') | \
                    (cells['RNA family'] == 'ET') |(cells['RNA family'] == 'NP') | (cells['RNA family'] == 'PT') |\
                    (cells['Cell'] == '20180115_sample_6' ) | (cells['Cell'] == '20180208_sample_1' ) |\
                    (cells['Cell'] == '20180315_sample_2' ) | (cells['Cell'] == '20180509_sample_1' ) | \
                    (cells['Cell'] == '20180828_sample_7' ) | (cells['Cell'] == '20190722_sample_7' )



inhibitory_index = (cells['RNA family'] == 'Lamp5') | (cells['RNA family'] == 'Pvalb') | \
                    (cells['RNA family'] == 'Sncg') | (cells['RNA family'] == 'Sst') | \
                    (cells['RNA family'] == 'Vip') | (cells['Cell'] == '20190606_sample_7') | \
                    (cells['Cell'] == '20190905_sample_1')

## Excitatory cells

In [133]:
# get only pyramidal cells
pyramidal_cells = morphometrics.set_index('cell id').loc[cells[excitatory_index]['Cell'].values]

In [134]:
idx_exc_morphometrics = copy.copy(full_idx)

# remove features that are not computed for more than 100 cells in the dataset
indices, counts = np.unique(np.where(pyramidal_cells[full_idx].isnull())[1], return_counts=True)
to_remove = [idx_exc_morphometrics[z] for z in indices[counts>100]]

# remove depth features because of the flattening of biocytin stainings in this direction
to_remove +=['axon robust height', 'axon robust width', 'axon robust depth',
             'dendrite depth', 'dendrite robust depth']
for z in set(to_remove):
    print('deleting %s'%z)
    idx_exc_morphometrics.remove(z)

deleting Log1p fraction of dendrite below axon
deleting axon depth
deleting dendrite depth
deleting Log1p fraction of dendrite above axon
deleting axon total length
deleting axon max branch angle
deleting axon tips
deleting axon mean neurite radius
deleting mean initial segment radius
deleting axon log min tortuosity
deleting axon log median tortuosity
deleting axon log max tortuosity
deleting axon robust height
deleting Log1p fraction of axon below dendrite
deleting axon max branch order
deleting axon branch points
deleting axon mean branch angle
deleting axon robust depth
deleting axon max path distance to soma
deleting EMD axon dendrite
deleting axon median path angle
deleting axon x-bias
deleting axon max path angle
deleting axon first bifurcation moment
deleting axon bifurcation standard deviation
deleting Log1p fraction of axon above dendrite
deleting axon height
deleting axon width
deleting axon max segment length
deleting axon z-bias
deleting axon robust width
deleting axon abo

In [135]:
pyramidal_cells = pyramidal_cells[idx_exc_morphometrics]

### coefficient of variation 

$c_v = \frac{\sigma}{\mu}$ 

Exclude everything that has $c_v < .25$

In [136]:
# morphometrics to be excluded due to little variation
cv_threshold = .25
np.array(idx_exc_morphometrics)[(pyramidal_cells.abs().std()/pyramidal_cells.abs().mean() < cv_threshold).values]

array(['dendrite above soma', 'dendrite log median tortuosity',
       'dendrite max branch angle', 'dendrite max path angle',
       'dendrite mean branch angle'], dtype='<U43')

In [137]:
idx_after_cv = np.array(idx_exc_morphometrics)[(pyramidal_cells.abs().std()/pyramidal_cells.abs().mean() >= cv_threshold).values]
print('remaining features: ', idx_after_cv, ' \n number of features: ', len(idx_after_cv))

remaining features:  ['"apical" branch points' '"apical" height'
 '"apical" log1p number of outer bifurcations'
 '"apical" mean bifurcation distance' '"apical" robust height'
 '"apical" robust width' '"apical" std bifurcation distance'
 '"apical" total length' '"apical" width'
 'dendrite bifurcation standard deviation' 'dendrite branch points'
 'dendrite first bifurcation moment' 'dendrite height'
 'dendrite log max tortuosity' 'dendrite log min tortuosity'
 'dendrite max Euclidean dist' 'dendrite max branch order'
 'dendrite max path distance to soma' 'dendrite max segment length'
 'dendrite mean neurite radius' 'dendrite median path angle'
 'dendrite min branch angle' 'dendrite robust height'
 'dendrite robust width' 'dendrite tips' 'dendrite total length'
 'dendrite width' 'dendrite x-bias' 'dendrite z-bias' 'normalized depth'
 'soma radius' 'stems' 'stems exiting down' 'stems exiting to the sides'
 'stems exiting up']  
 number of features:  35


In [138]:
final_exc_idx = idx_after_cv
print('Final number of excitatory features: ', len(final_exc_idx))

Final number of excitatory features:  35


## Inhibitory cells

In [139]:
# get only inhibitory cells
inhibitory_cells = morphometrics.set_index('cell id').loc[cells[inhibitory_index]['Cell'].values]
len(full_idx)

75

In [140]:
idx_inh_morphometrics = copy.copy(full_idx)
indices, counts = np.unique(np.where(inhibitory_cells[full_idx].isnull())[1], return_counts=True)

to_remove = [idx_inh_morphometrics[z] for z in indices[counts> 100]]
to_remove += [k for k in morphometrics.columns if k.find('"apical"')>-1 ]
to_remove += ['axon depth', 'axon robust depth', 'dendrite depth', 'dendrite robust depth']
for z in set(to_remove):
    print('deleting %s'%z)
    idx_inh_morphometrics.remove(z)

deleting "apical" log1p number of outer bifurcations
deleting axon depth
deleting dendrite depth
deleting "apical" std bifurcation distance
deleting axon robust depth
deleting "apical" mean bifurcation distance
deleting dendrite robust depth
deleting "apical" branch points
deleting "apical" robust height
deleting "apical" height
deleting "apical" width
deleting "apical" robust width
deleting "apical" total length


In [141]:
inhibitory_cells = inhibitory_cells[idx_inh_morphometrics]

# morphometrics to be excluded due to little variation
print('Features excluded due to little variation: \n', np.array(idx_inh_morphometrics)[(inhibitory_cells.abs().std()/inhibitory_cells.abs().mean() < 0.25).values])

idx_after_cv = np.array(idx_inh_morphometrics)[(inhibitory_cells.abs().std()/inhibitory_cells.abs().mean() >= 0.25).values]
print('remaining features: \n', idx_after_cv, ' \n number of features: ', len(idx_after_cv))

Features excluded due to little variation: 
 ['axon log max tortuosity' 'axon log median tortuosity'
 'axon max branch angle' 'axon max path angle' 'axon mean branch angle'
 'axon median path angle' 'dendrite max path angle'
 'dendrite mean branch angle' 'dendrite median path angle']
remaining features: 
 ['EMD axon dendrite' 'Log1p fraction of axon above dendrite'
 'Log1p fraction of axon below dendrite'
 'Log1p fraction of dendrite above axon'
 'Log1p fraction of dendrite below axon' 'axon above soma'
 'axon bifurcation standard deviation' 'axon branch points'
 'axon first bifurcation moment' 'axon height' 'axon log min tortuosity'
 'axon max Euclidean dist' 'axon max branch order'
 'axon max path distance to soma' 'axon max segment length'
 'axon mean neurite radius' 'axon min branch angle' 'axon robust height'
 'axon robust width' 'axon tips' 'axon total length' 'axon width'
 'axon x-bias' 'axon z-bias' 'dendrite above soma'
 'dendrite bifurcation standard deviation' 'dendrite bran

In [142]:
final_inh_idx = list(idx_after_cv)

# exclude features after visual inspection
to_remove = [ 'Log1p fraction of dendrite below axon',
             'dendrite max branch angle', 'dendrite min branch angle']

for z in to_remove:
    final_inh_idx.remove(z)

print('Final number of inhibitory features: ', len(final_inh_idx))

Final number of inhibitory features:  50


# Concatenate and store the features

## Morphometric statistics

In [148]:
e_morph = pyramidal_cells[final_exc_idx]
e_morph.loc[:,'cell class'] = 'exc'

i_morph = inhibitory_cells[final_inh_idx]
i_morph.loc[:,'cell class'] = 'inh'

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [150]:
morphometric_used = pd.concat((e_morph,i_morph))
morphometric_used.to_csv('../data/m1_patchseq_morph_features.csv')

## z-profiles

In [152]:
excitatory_cells = morphometrics.set_index('cell id').loc[cells[excitatory_index]['Cell'].values]


profile_dict_e= dict()
for rn,item in list(excitatory_cells.iterrows()):
    profile = item['dendrite z-profile']
    if profile:
        s = profile.replace('\n', '').replace('[', '').replace(']','')
        no = [x for x in s.split(' ') if x != '']
        profile_dict_e[rn] = np.array([float(n) for n in no])

In [155]:
inhibitory_cells = morphometrics.set_index('cell id').loc[cells[inhibitory_index]['Cell'].values]
    
# first load them assigned to their index/name. Then put them in an array. To make sure they correspond. 
profile_dict_i=dict()
for rn,item in list(inhibitory_cells.iterrows()):
    profile_dict_i[rn] = np.array([])
    for profile in item[['axon z-profile']]:
        if profile is not np.nan:
            s = profile.replace('\n', '').replace('[', '').replace(']','')
            no = [x for x in s.split(' ') if x != '']
            temp = np.array([float(n) for n in no])
        else:
            temp = np.zeros((1,20))
        
        if profile_dict_i[rn].size == 0:
            profile_dict_i[rn] = temp
        else:  
            profile_dict_i[rn] = np.vstack((profile_dict_i[rn],temp))


In [156]:
z_profiles = (pd.DataFrame(profile_dict_e).T).append(pd.DataFrame(profile_dict_i).T)
z_profiles = z_profiles.reset_index().rename(columns={'index':'cell id'}).set_index('cell id')
z_profiles.to_csv("../data/m1_patchseq_morph_zprofiles.csv")